In [ ]:
# File: cards.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
from frontend import App

# create an app
app = App.create("cards")

# create a rectangular card mesh
mesh_res, n_stack, card_height = 8, 8, 0.25
card_width = 0.75 * card_height
V, F = app.mesh.rectangle(mesh_res, card_width, card_height, [0, 0, 1], [0, 1, 0])
app.asset.add.tri("card", V, F)

# create a sphere mesh to knock down the house of cards
V, F, T = app.mesh.icosphere(r=0.15, subdiv_count=3).tetrahedralize()
app.asset.add.tet("sphere", V, F, T)

# create a scene
scene = app.scene.create()
angle, gap = 25.0, 1e-3


# function to build a row of card pairs with ceiling cards on top
def make_row(n: int, _x: float, y: float, cards: list) -> float:
    x, _y, ceil_x = 0, 0, []
    for i in range(n):
        # add left card tilted at angle
        left = scene.add("card").rotate(-angle, "z")
        left.at(x - left.min("x") + (_x if i == 0 else 0), y - left.min("y"), 0)
        if i == 0:
            _x = left.max("x")
        # add right card tilted in opposite direction
        right = scene.add("card").rotate(angle, "z")
        shift = gap + left.max("x") - right.min("x")
        right.at(shift, y - right.min("y"), 0)
        if i < n - 1:
            ceil_x.append(right.max("x"))
        x = right.max("x") + gap
        max_y = right.max("y") + gap
        cards.extend([left, right])
    # add horizontal ceiling cards
    for i, x in enumerate(ceil_x):
        z = max_y if i % 2 == 0 else max_y + gap
        ceil = scene.add("card").rotate(-90, "z").at(x, z, 0)
        _y = max(_y, ceil.max("y"))
        cards.append(ceil)
    return _x, _y + gap


# build the house of cards pyramid
_x, _y, _cards = -0.75, gap, []
for i in reversed(range(n_stack)):
    _x, _y = make_row(i + 1, _x, _y, _cards)

# set material properties for all cards
for card in _cards:
    (
        card.param.set("contact-gap", gap)
        .set("young-mod", 30000)
        .set("bend", 1e6)
        .set("friction", 0.5)
    )

# add sphere projectile with initial velocity
scene.add("sphere").at(-2, 1, 0).jitter().velocity(2.3, 0, 0)

# add invisible floor
scene.add.invisible.wall([0, 0, 0], [0, 1, 0])

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview()

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters
session.param.set("dt", 0.01).set("min-newton-steps", 32)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview()

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate()

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()